In [1]:
%pip install pyautogen llama-index llama-index-tools-wikipedia llama-index-readers-wikipedia wikipedia

Note: you may need to restart the kernel to use updated packages.


In [18]:
!pip install gradio

  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached starlette-0.37.2-py3-none-any.whl.metadata (5.9 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.6 MB 10.7 MB/s eta 0:00:01
   --------------- ------------------------ 4.7/12.6 MB 11.9 MB/s eta 0:00:01
   --------------------

In [10]:
from llama_index.core import Settings
from llama_index.core.agent import ReActAgent
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.tools.wikipedia import WikipediaToolSpec
from autogen.agentchat.contrib.llamaindex_conversable_agent import LLamaIndexConversableAgent
import os
import autogen
import dotenv

In [3]:
dotenv.load_dotenv()

True

In [4]:
# config_list = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={"tags": ["gpt-3.5-turbo"]},  # comment out to get all
# )


# When using a single openai endpoint, you can use the following:
config_list = [{"model": "gpt-4o-mini", "api_key": os.getenv("OPENAI_API_KEY")}]

In [5]:
llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [6]:
embed_model = OpenAIEmbedding(
    model="text-embedding-ada-002",
    temperature=0.0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [7]:
Settings.llm = llm
Settings.embed_model = embed_model

In [8]:
# create a react agent to use wikipedia tool
wiki_spec = WikipediaToolSpec()
# Get the search wikipedia tool
wikipedia_tool = wiki_spec.to_tool_list()[1]

In [9]:
location_specialist = ReActAgent.from_tools(tools=[wikipedia_tool], llm=llm, max_iterations=10, verbose=True)

In [11]:
llm_config = {
    "temperature": 0,
    "config_list": config_list,
}

In [12]:
trip_assistant = LLamaIndexConversableAgent(
    "trip_specialist",
    llama_index_agent=location_specialist,
    system_message="You help customers finding more about places they would like to visit. You can use external resources to provide more details as you engage with the customer.",
    description="This agents helps customers discover locations to visit, things to do, and other details about a location. It can use external resources to provide more details. This agent helps in finding attractions, history and all that there is to know about a place",

)

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    human_input_mode="ALWAYS",
    code_execution_config=False,
)

In [14]:
groupchat = autogen.GroupChat(
    agents=[trip_assistant, user_proxy],
    messages=[],
    max_round=500,
    speaker_selection_method="round_robin",
    enable_clear_history=True,
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config,)

In [21]:
chat_result = user_proxy.initiate_chat(
    manager,
    message="""
Can you plan 2 day trip to New Delhi, India
""",
)

Admin (to chat_manager):


Can you plan 2 day trip to New Delhi, India


--------------------------------------------------------------------------------

Next speaker: trip_specialist


>>>>>>>> USING AUTO REPLY...
> Running step e1308d41-c6b9-494a-8955-8f63263fe98a. Step input: 
Can you plan 2 day trip to New Delhi, India

Thought: The current language of the user is: English. I can provide a plan for a 2-day trip to New Delhi, India without needing to use a tool.
Answer: Here’s a suggested itinerary for a 2-day trip to New Delhi, India:

### Day 1: Historical and Cultural Exploration
- **Morning:**
  - **Red Fort (Lal Qila)**: Start your day by visiting this UNESCO World Heritage Site. Explore the stunning architecture and learn about its history.
  - **Jama Masjid**: Just a short walk from the Red Fort, this is one of the largest mosques in India. Climb the minaret for a panoramic view of the city.

- **Lunch:**
  - Enjoy some local cuisine at a nearby restaurant in Chandni Chowk. 